In [1]:
# this notebook converts the CSV to ES mapping

In [2]:
# imports
from datetime import datetime
import hashlib
import json
import sys
import csv
import os
import pandas as pd
import re
import time
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords

In [3]:
# some long text
# source: https://www.kaggle.com/c/stanford-covid-vaccine
text1 = '''
Winning the fight against the COVID-19 pandemic will require an effective vaccine that can be equitably and widely distributed. Building upon decades of research has allowed scientists to accelerate the search for a vaccine against COVID-19, but every day that goes by without a vaccine has enormous costs for the world nonetheless. We need new, fresh ideas from all corners of the world. Could online gaming and crowdsourcing help solve a worldwide pandemic? Pairing scientific and crowdsourced intelligence could help computational biochemists make measurable progress.
mRNA vaccines have taken the lead as the fastest vaccine candidates for COVID-19, but currently, they face key potential limitations. One of the biggest challenges right now is how to design super stable messenger RNA molecules (mRNA). Conventional vaccines (like your seasonal flu shots) are packaged in disposable syringes and shipped under refrigeration around the world, but that is not currently possible for mRNA vaccines.
Researchers have observed that RNA molecules have the tendency to spontaneously degrade. This is a serious limitation--a single cut can render the mRNA vaccine useless. Currently, little is known on the details of where in the backbone of a given RNA is most prone to being affected. Without this knowledge, current mRNA vaccines against COVID-19 must be prepared and shipped under intense refrigeration, and are unlikely to reach more than a tiny fraction of human beings on the planet unless they can be stabilized.
The Eterna community, led by Professor Rhiju Das, a computational biochemist at Stanford’s School of Medicine, brings together scientists and gamers to solve puzzles and invent medicine. Eterna is an online video game platform that challenges players to solve scientific problems such as mRNA design through puzzles. The solutions are synthesized and experimentally tested at Stanford by researchers to gain new insights about RNA molecules. The Eterna community has previously unlocked new scientific principles, made new diagnostics against deadly diseases, and engaged the world’s most potent intellectual resources for the betterment of the public. The Eterna community has advanced biotechnology through its contribution in over 20 publications, including advances in RNA biotechnology.
In this competition, we are looking to leverage the data science expertise of the Kaggle community to develop models and design rules for RNA degradation. Your model will predict likely degradation rates at each base of an RNA molecule, trained on a subset of an Eterna dataset comprising over 3000 RNA molecules (which span a panoply of sequences and structures) and their degradation rates at each position. We will then score your models on a second generation of RNA sequences that have just been devised by Eterna players for COVID-19 mRNA vaccines. These final test sequences are currently being synthesized and experimentally characterized at Stanford University in parallel to your modeling efforts -- Nature will score your models!
Improving the stability of mRNA vaccines was a problem that was being explored before the pandemic but was expected to take many years to solve. Now, we must solve this deep scientific challenge in months, if not weeks, to accelerate mRNA vaccine research and deliver a refrigerator-stable vaccine against SARS-CoV-2, the virus behind COVID-19. The problem we are trying to solve has eluded academic labs, industry R&D groups, and supercomputers, and so we are turning to you. To help, you can join the team of video game players, scientists, and developers at Eterna to unlock the key in our fight against this devastating pandemic. 
'''

# and a short one
text2 = 'The quick brown fox jumps over the lazy dog'

In [4]:
# function to count words
def word_count(text):
    if isinstance(text, str):
        s = text.split(' ')
        return len(s)
    else:
        return 0

print('words:', word_count(text1))
print('words:', word_count(text2))
print('words:', word_count(None))

words: 564
words: 9
words: 0


In [5]:
# function to count sentences
def sentence_count(text):
    if isinstance(text, str):
        s = text.split('. ')
        return len(s)
    else:
        return 0

print('sentences:', sentence_count(text1))
print('sentences:', sentence_count(text2))
print('sentences:', sentence_count(None))

sentences: 20
sentences: 1
sentences: 0


In [6]:
# extractive summarization

In [7]:
# text summarization 100% -> n%
def nltk_ratio(text, ratio=0.25):
    return summarize(text, ratio=ratio)

sum_nltk_ratio = nltk_ratio(text1, ratio=0.25)
print('words:', word_count(sum_nltk_ratio))
print(sum_nltk_ratio)

words: 139
Eterna is an online video game platform that challenges players to solve scientific problems such as mRNA design through puzzles.
The solutions are synthesized and experimentally tested at Stanford by researchers to gain new insights about RNA molecules.
We will then score your models on a second generation of RNA sequences that have just been devised by Eterna players for COVID-19 mRNA vaccines.
Improving the stability of mRNA vaccines was a problem that was being explored before the pandemic but was expected to take many years to solve.
Now, we must solve this deep scientific challenge in months, if not weeks, to accelerate mRNA vaccine research and deliver a refrigerator-stable vaccine against SARS-CoV-2, the virus behind COVID-19.
To help, you can join the team of video game players, scientists, and developers at Eterna to unlock the key in our fight against this devastating pandemic.


In [8]:
# text summarization 100% -> n words
def nltk_count(text, word_count=100):
    return summarize(text, word_count=word_count)

sum_nltk_count = nltk_count(text1, word_count=100)
print('words:', word_count(sum_nltk_count))
print(sum_nltk_count)

words: 98
Eterna is an online video game platform that challenges players to solve scientific problems such as mRNA design through puzzles.
We will then score your models on a second generation of RNA sequences that have just been devised by Eterna players for COVID-19 mRNA vaccines.
Now, we must solve this deep scientific challenge in months, if not weeks, to accelerate mRNA vaccine research and deliver a refrigerator-stable vaccine against SARS-CoV-2, the virus behind COVID-19.
To help, you can join the team of video game players, scientists, and developers at Eterna to unlock the key in our fight against this devastating pandemic.


In [9]:
# adaptive summarization
# https://www.machinelearningplus.com/nlp/text-summarization-approaches-nlp-example/

In [10]:
# BART
# Importing the model
from transformers import BartForConditionalGeneration, BartTokenizer, BartConfig

In [11]:
'''
# Loading the model and tokenizer for bart-large-cnn
tokenizer=BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model=BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
#'''

"\n# Loading the model and tokenizer for bart-large-cnn\ntokenizer=BartTokenizer.from_pretrained('facebook/bart-large-cnn')\nmodel=BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')\n#"

In [12]:
'''
# Encoding the inputs and passing them to model.generate()
def bart(text):
    inputs = tokenizer.batch_encode_plus([text],return_tensors='pt')
    summary_ids = model.generate(inputs['input_ids'], early_stopping=True)

    # Decoding and printing the summary
    bart_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    return bart_summary

# long text
start = time.time()
sum_bart_l = bart(text1)
end = time.time()

print('### long text ###')
print('runtime:', end-start)
print('words:', word_count(sum_bart_l))
print('sentences:', sentence_count(sum_bart_l))
print(sum_bart_l)
print('')

# short text
print('### short text ###')
start = time.time()
sum_bart_s = bart(text2)
end = time.time()

print('runtime:', end-start)
print('words:', word_count(sum_bart_s))
print('sentences:', sentence_count(sum_bart_s))
print(sum_bart_s)
#'''

"\n# Encoding the inputs and passing them to model.generate()\ndef bart(text):\n    inputs = tokenizer.batch_encode_plus([text],return_tensors='pt')\n    summary_ids = model.generate(inputs['input_ids'], early_stopping=True)\n\n    # Decoding and printing the summary\n    bart_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)\n    \n    return bart_summary\n\n# long text\nstart = time.time()\nsum_bart_l = bart(text1)\nend = time.time()\n\nprint('### long text ###')\nprint('runtime:', end-start)\nprint('words:', word_count(sum_bart_l))\nprint('sentences:', sentence_count(sum_bart_l))\nprint(sum_bart_l)\nprint('')\n\n# short text\nprint('### short text ###')\nstart = time.time()\nsum_bart_s = bart(text2)\nend = time.time()\n\nprint('runtime:', end-start)\nprint('words:', word_count(sum_bart_s))\nprint('sentences:', sentence_count(sum_bart_s))\nprint(sum_bart_s)\n#"

In [13]:
# T5
# https://towardsdatascience.com/summarize-reddit-comments-using-t5-bart-gpt-2-xlnet-models-a3e78a5ab944
from transformers import T5Tokenizer, T5ForConditionalGeneration
model = T5ForConditionalGeneration.from_pretrained('t5-base')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

Some weights of the model checkpoint at t5-base were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
def t5(text):
    Preprocessed_text = "summarize: " + text
    tokens_input = tokenizer.encode(Preprocessed_text,return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(tokens_input, min_length=100, max_length=180, length_penalty=4.0)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

'''
# long text
start = time.time()
sum_t5_l = t5(text1)
end = time.time()

print('### long text ###')
print('runtime:', end-start)
print('words:', word_count(sum_t5_l))
print('sentences:', sentence_count(sum_t5_l))
print(sum_t5_l)
print('')

# short text
print('### short text ###')
start = time.time()
sum_t5_s = t5(text2)
end = time.time()

print('runtime:', end-start)
print('words:', word_count(sum_t5_s))
print('sentences:', sentence_count(sum_t5_s))
print(sum_t5_s)
#'''

"\n# long text\nstart = time.time()\nsum_t5_l = t5(text1)\nend = time.time()\n\nprint('### long text ###')\nprint('runtime:', end-start)\nprint('words:', word_count(sum_t5_l))\nprint('sentences:', sentence_count(sum_t5_l))\nprint(sum_t5_l)\nprint('')\n\n# short text\nprint('### short text ###')\nstart = time.time()\nsum_t5_s = t5(text2)\nend = time.time()\n\nprint('runtime:', end-start)\nprint('words:', word_count(sum_t5_s))\nprint('sentences:', sentence_count(sum_t5_s))\nprint(sum_t5_s)\n#"

In [15]:
# industry categories

# https://www.census.gov/programs-surveys/aces/information/iccl.html
cat_sic = ['Agriculture','Forestry','Fishing','Mining','Construction','Manufacturing','Transportation','Communications','Electric','Gas','Sanitary','Wholesale Trade','Retail Trade','Finance','Insurance','Real Estate','Services','Public Administration']
# https://www.marketing91.com/19-types-of-business-industries/
cat_19 = ['Aerospace','Transport','Computer','Telecommunication','Agriculture','Construction','Education','Pharmaceutical','Food','Health care','Hospitality','Entertainment','News Media','Energy','Manufacturing','Music','Mining','Worldwide web','Electronics']
# https://simplicable.com/new/industries
cat_simple = ['Advertising','Agriculture','Communication','Construction','Creative','Education','Entertainment','Fashion','Finance','Health care','Information Technology','Manufacturing','Media','Retail','Research','Robotics','Space']

cat = ['Accommodation & Food','Accounting','Agriculture','Banking & Insurance','Biotechnological & Life Sciences','Construction & Engineering','Economics','Education & Research','Emergency & Relief','Finance','Government and Public Works','Healthcare','Justice, Law and Regulations','Manufacturing','Media & Publishing','Miscellaneous','Physics','Real Estate, Rental & Leasing','Utilities','Wholesale & Retail']
subcat = ['Failure','Food','Fraud','General','Genomics','Insurance and Risk','Judicial Applied','Life-sciences','Machine Learning','Maintenance','Management and Operations','Marketing','Material Science','Physical','Policy and Regulatory','Politics','Preventative and Reactive','Quality','Real Estate','Rental & Leasing','Restaurant','Retail','School','Sequencing','Social Policies','Student','Textual Analysis','Tools','Tourism','Trading & Investment','Transportation','Valuation','Water & Pollution','Wholesale']

In [16]:
# zero shot classification
# https://towardsdatascience.com/zero-shot-text-classification-with-hugging-face-7f533ba83cd6
from transformers import pipeline
classifier = pipeline("zero-shot-classification")

Some weights of the model checkpoint at facebook/bart-large-mnli were not used when initializing BartModel: ['model.encoder.version', 'model.decoder.version']
- This IS expected if you are initializing BartModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BartModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at facebook/bart-large-mnli were not used when initializing BartForSequenceClassification: ['model.encoder.version', 'model.decoder.version']
- This IS expected if you are initializing BartForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification m

In [17]:
'''
# test classifictaion with nltk (200 words)

s = nltk_count(text1, word_count=100)

start = time.time()
#res = classifier(nltk_count(text1, word_count=200), cat_19)
res_sic = classifier(s, cat_sic, multi_class=True)
end = time.time()

print('runtime sic:', end-start)
#print(res_sic)
print(res_sic['labels'][0:3])
print(res_sic['scores'][0:3])


start = time.time()
#res = classifier(nltk_count(text1, word_count=200), cat_19)
res_c19 = classifier(s, cat_19, multi_class=True)
end = time.time()

print('runtime c19:', end-start)
#print(res_c19)
print(res_c19['labels'][0:3])
print(res_c19['scores'][0:3])


start = time.time()
#res = classifier(nltk_count(text1, word_count=200), cat_19)
res_simple = classifier(s, cat_simple, multi_class=True)
end = time.time()

print('runtime simple:', end-start)
#print(res_simple)
print(res_simple['labels'][0:3])
print(res_simple['scores'][0:3])


start = time.time()
#res = classifier(nltk_count(text1, word_count=200), cat)
res_simple = classifier(s, cat, multi_class=True)
end = time.time()

print('runtime category:', end-start)
#print(res_simple)
print(res_simple['labels'][0:3])
print(res_simple['scores'][0:3])


start = time.time()
#res = classifier(nltk_count(text1, word_count=200), subcat)
res_simple = classifier(s, subcat, multi_class=True)
end = time.time()

print('runtime subcategory:', end-start)
#print(res_simple)
print(res_simple['labels'][0:3])
print(res_simple['scores'][0:3])
#'''

"\n# test classifictaion with nltk (200 words)\n\ns = nltk_count(text1, word_count=100)\n\nstart = time.time()\n#res = classifier(nltk_count(text1, word_count=200), cat_19)\nres_sic = classifier(s, cat_sic, multi_class=True)\nend = time.time()\n\nprint('runtime sic:', end-start)\n#print(res_sic)\nprint(res_sic['labels'][0:3])\nprint(res_sic['scores'][0:3])\n\n\nstart = time.time()\n#res = classifier(nltk_count(text1, word_count=200), cat_19)\nres_c19 = classifier(s, cat_19, multi_class=True)\nend = time.time()\n\nprint('runtime c19:', end-start)\n#print(res_c19)\nprint(res_c19['labels'][0:3])\nprint(res_c19['scores'][0:3])\n\n\nstart = time.time()\n#res = classifier(nltk_count(text1, word_count=200), cat_19)\nres_simple = classifier(s, cat_simple, multi_class=True)\nend = time.time()\n\nprint('runtime simple:', end-start)\n#print(res_simple)\nprint(res_simple['labels'][0:3])\nprint(res_simple['scores'][0:3])\n\n\nstart = time.time()\n#res = classifier(nltk_count(text1, word_count=200),

In [18]:
'''
# test classifictaion with t5
start = time.time()
#res = classifier(nltk_count(text1, word_count=200), cat_19)
res_sic = classifier(sum_t5_l, cat_sic, multi_class=True)
end = time.time()

print('runtime sic:', end-start)
#print(res_sic)
print(res_sic['labels'][0:3])
print(res_sic['scores'][0:3])


start = time.time()
#res = classifier(nltk_count(text1, word_count=200), cat_19)
res_c19 = classifier(sum_t5_l, cat_19, multi_class=True)
end = time.time()

print('runtime c19:', end-start)
#print(res_c19)
print(res_c19['labels'][0:3])
print(res_c19['scores'][0:3])


start = time.time()
#res = classifier(nltk_count(text1, word_count=200), cat_19)
res_simple = classifier(sum_t5_l, cat_simple, multi_class=True)
end = time.time()

print('runtime simple:', end-start)
#print(res_simple)
print(res_simple['labels'][0:3])
print(res_simple['scores'][0:3])


start = time.time()
#res = classifier(nltk_count(text1, word_count=200), cat)
res_simple = classifier(sum_t5_l, cat, multi_class=True)
end = time.time()

print('runtime category:', end-start)
#print(res_simple)
print(res_simple['labels'][0:3])
print(res_simple['scores'][0:3])


start = time.time()
#res = classifier(nltk_count(text1, word_count=200), subcat)
res_simple = classifier(sum_t5_l, subcat, multi_class=True)
end = time.time()

print('runtime subcategory:', end-start)
#print(res_simple)
print(res_simple['labels'][0:3])
print(res_simple['scores'][0:3])
#'''

"\n# test classifictaion with t5\nstart = time.time()\n#res = classifier(nltk_count(text1, word_count=200), cat_19)\nres_sic = classifier(sum_t5_l, cat_sic, multi_class=True)\nend = time.time()\n\nprint('runtime sic:', end-start)\n#print(res_sic)\nprint(res_sic['labels'][0:3])\nprint(res_sic['scores'][0:3])\n\n\nstart = time.time()\n#res = classifier(nltk_count(text1, word_count=200), cat_19)\nres_c19 = classifier(sum_t5_l, cat_19, multi_class=True)\nend = time.time()\n\nprint('runtime c19:', end-start)\n#print(res_c19)\nprint(res_c19['labels'][0:3])\nprint(res_c19['scores'][0:3])\n\n\nstart = time.time()\n#res = classifier(nltk_count(text1, word_count=200), cat_19)\nres_simple = classifier(sum_t5_l, cat_simple, multi_class=True)\nend = time.time()\n\nprint('runtime simple:', end-start)\n#print(res_simple)\nprint(res_simple['labels'][0:3])\nprint(res_simple['scores'][0:3])\n\n\nstart = time.time()\n#res = classifier(nltk_count(text1, word_count=200), cat)\nres_simple = classifier(sum_t

In [19]:
'''
# test classifictaion with bart

start = time.time()
#res = classifier(nltk_count(text1, word_count=200), cat_19)
res_sic = classifier(sum_bart_l, cat_sic, multi_class=True)
end = time.time()

print('runtime sic:', end-start)
#print(res_sic)
print(res_sic['labels'][0:3])
print(res_sic['scores'][0:3])


start = time.time()
#res = classifier(nltk_count(text1, word_count=200), cat_19)
res_c19 = classifier(sum_bart_l, cat_19, multi_class=True)
end = time.time()

print('runtime c19:', end-start)
#print(res_c19)
print(res_c19['labels'][0:3])
print(res_c19['scores'][0:3])


start = time.time()
#res = classifier(nltk_count(text1, word_count=200), cat_19)
res_simple = classifier(sum_bart_l, cat_simple, multi_class=True)
end = time.time()

print('runtime simple:', end-start)
#print(res_simple)
print(res_simple['labels'][0:3])
print(res_simple['scores'][0:3])


start = time.time()
#res = classifier(nltk_count(text1, word_count=200), cat)
res_simple = classifier(sum_bart_l, cat, multi_class=True)
end = time.time()

print('runtime category:', end-start)
#print(res_simple)
print(res_simple['labels'][0:3])
print(res_simple['scores'][0:3])


start = time.time()
#res = classifier(nltk_count(text1, word_count=200), subcat)
res_simple = classifier(sum_bart_l, subcat, multi_class=True)
end = time.time()

print('runtime subcategory:', end-start)
#print(res_simple)
print(res_simple['labels'][0:3])
print(res_simple['scores'][0:3])
#'''

"\n# test classifictaion with bart\n\nstart = time.time()\n#res = classifier(nltk_count(text1, word_count=200), cat_19)\nres_sic = classifier(sum_bart_l, cat_sic, multi_class=True)\nend = time.time()\n\nprint('runtime sic:', end-start)\n#print(res_sic)\nprint(res_sic['labels'][0:3])\nprint(res_sic['scores'][0:3])\n\n\nstart = time.time()\n#res = classifier(nltk_count(text1, word_count=200), cat_19)\nres_c19 = classifier(sum_bart_l, cat_19, multi_class=True)\nend = time.time()\n\nprint('runtime c19:', end-start)\n#print(res_c19)\nprint(res_c19['labels'][0:3])\nprint(res_c19['scores'][0:3])\n\n\nstart = time.time()\n#res = classifier(nltk_count(text1, word_count=200), cat_19)\nres_simple = classifier(sum_bart_l, cat_simple, multi_class=True)\nend = time.time()\n\nprint('runtime simple:', end-start)\n#print(res_simple)\nprint(res_simple['labels'][0:3])\nprint(res_simple['scores'][0:3])\n\n\nstart = time.time()\n#res = classifier(nltk_count(text1, word_count=200), cat)\nres_simple = classi

In [20]:
# category function
def categorize(text, categories, first=True, treshold=0):
    start = time.time()
    res = classifier(text, categories, multi_class=True)
    #print(res)
    end = time.time()
    dur = round(end-start, 3)
    if first == True:
        return {
            'category': res['labels'][0],
            'score': res['scores'][0],
            'runtime': dur,
        } if res['scores'][0] >= treshold else {
            'category': None,
            'score': None,
            'runtime': dur,
        }
    else:
        ret = dict(zip(res['labels'], res['scores']))
        ret = {key: val for key, val in filter(lambda sub: sub[1] >= treshold, ret.items())} 
        return ret
        

#print(categorize(nltk_count(text1), cat, first=False, treshold=0.5))
#print(categorize(nltk_count(text1), cat, first=True, treshold=0.9))

In [21]:
# language detection
# https://towardsdatascience.com/how-to-detect-and-translate-languages-for-nlp-project-dfd52af0c3b5
from langdetect import detect, detect_langs, DetectorFactory

language_codes = {'af': 'afrikaans', 'sq': 'albanian', 'am': 'amharic', 'ar': 'arabic', 'hy': 'armenian', 'az': 'azerbaijani', 'eu': 'basque', 'be': 'belarusian', 'bn': 'bengali', 'bs': 'bosnian', 'bg': 'bulgarian', 'ca': 'catalan', 'ceb': 'cebuano', 'ny': 'chichewa', 'zh-cn': 'chinese (simplified)', 'zh-tw': 'chinese (traditional)', 'co': 'corsican', 'hr': 'croatian', 'cs': 'czech', 'da': 'danish', 'nl': 'dutch', 'en': 'english', 'eo': 'esperanto', 'et': 'estonian', 'tl': 'filipino', 'fi': 'finnish', 'fr': 'french', 'fy': 'frisian', 'gl': 'galician', 'ka': 'georgian', 'de': 'german', 'el': 'greek', 'gu': 'gujarati', 'ht': 'haitian creole', 'ha': 'hausa', 'haw': 'hawaiian', 'iw': 'hebrew', 'hi': 'hindi', 'hmn': 'hmong', 'hu': 'hungarian', 'is': 'icelandic', 'ig': 'igbo', 'id': 'indonesian', 'ga': 'irish', 'it': 'italian', 'ja': 'japanese', 'jw': 'javanese', 'kn': 'kannada', 'kk': 'kazakh', 'km': 'khmer', 'ko': 'korean', 'ku': 'kurdish (kurmanji)', 'ky': 'kyrgyz', 'lo': 'lao', 'la': 'latin', 'lv': 'latvian', 'lt': 'lithuanian', 'lb': 'luxembourgish', 'mk': 'macedonian', 'mg': 'malagasy', 'ms': 'malay', 'ml': 'malayalam', 'mt': 'maltese', 'mi': 'maori', 'mr': 'marathi', 'mn': 'mongolian', 'my': 'myanmar (burmese)', 'ne': 'nepali', 'no': 'norwegian', 'ps': 'pashto', 'fa': 'persian', 'pl': 'polish', 'pt': 'portuguese', 'pa': 'punjabi', 'ro': 'romanian', 'ru': 'russian', 'sm': 'samoan', 'gd': 'scots gaelic', 'sr': 'serbian', 'st': 'sesotho', 'sn': 'shona', 'sd': 'sindhi', 'si': 'sinhala', 'sk': 'slovak', 'sl': 'slovenian', 'so': 'somali', 'es': 'spanish', 'su': 'sundanese', 'sw': 'swahili', 'sv': 'swedish', 'tg': 'tajik', 'ta': 'tamil', 'te': 'telugu', 'th': 'thai', 'tr': 'turkish', 'uk': 'ukrainian', 'ur': 'urdu', 'uz': 'uzbek', 'vi': 'vietnamese', 'cy': 'welsh', 'xh': 'xhosa', 'yi': 'yiddish', 'yo': 'yoruba', 'zu': 'zulu', 'fil': 'Filipino', 'he': 'Hebrew'}

def lingo(text, simple=True):
    DetectorFactory.seed = 0
    
    if simple == True:
        return detect(text) #language_codes[detect(text)]
    else:
        l = str(detect_langs(text)[0]).split(':')
        l = {
            'code': l[0],
            'language': language_codes[ l[0] ],
            'probability': l[1],
        }
        return l

sentence = "Tanzania ni nchi inayoongoza kwa utalii barani afrika"
sentence2 = "Heute schneit es."

print(lingo(sentence, simple=False))
print(lingo(sentence2))
print(lingo(text1))
print(lingo(text2))

{'code': 'sw', 'language': 'swahili', 'probability': '0.9999971210408874'}
de
en
en


In [22]:
# helper functions

In [23]:
# function to rebuild list from string
# that happens when it is stored in CSV without json-encode the data
def str_to_list(s):
    s = s.replace("'", "").replace(' ,', ',').replace(
        '[', '').replace(']', '').split(',')
    s = [i.replace('"','').strip() for i in s if i]
    return s

In [24]:
# helper function to create folder create_folder
def create_folder(path):
    if not os.path.exists(os.path.dirname(path)):
        try:
            os.makedirs(os.path.dirname(path))
            print(path + ' created')
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise

In [25]:
# generic store data to file function
def store_data(data, file, mode='w', toJson=False):
    if toJson:
        data = json.dumps(data)
    with open(file, mode, encoding='utf-8') as fp:
        result = fp.write(data)
        return result
    
# generic load data from file function
def load_data(file, fromJson=False):
    if os.path.isfile(file):
        with open(file, 'r', encoding='utf-8', errors="ignore") as fp:
            data = fp.read()
            if fromJson:
                data = json.loads(data)
            return data
    else:
        return 'file not found'

# test text
#print(store_data('Hello', '../data/repositories/mlart/test.txt'))
#print(load_data('../data/repositories/mlart/test.txt'))

# test json
#print(store_data({'msg':'Hello World'}, '../data/repositories/mlart/test.json', toJson=True))
#print(load_data('../data/repositories/mlart/test.json', fromJson=True))

#store_data(result[0]['html'], '../data/repositories/kaggle/notebook.html')
#store_data(result[0]['iframe'], '../data/repositories/kaggle/kernel.html')

In [37]:
# remove special characters
def clean_text(text):
    # Ref: https://gist.github.com/Alex-Just/e86110836f3f93fe7932290526529cd1#gistcomment-3208085
    # Ref: https://en.wikipedia.org/wiki/Unicode_block
    EMOJI_PATTERN = re.compile(
        "(["
        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F700-\U0001F77F"  # alchemical symbols
        "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
        "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
        "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        "\U0001FA00-\U0001FA6F"  # Chess Symbols
        "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        "\U00002702-\U000027B0"  # Dingbats
        "])"
    )
    text = re.sub(EMOJI_PATTERN, '', text)
    
    # additional cleanup
    text = text.replace('•','').replace('\n',' ')
    
    return text

In [42]:
tag_filter = {
    # '3D',
    '3D Ken Burns Effect': 'Ken Burns 3D',
    # '3D Photo Inpainting',
    'AI': None,
    # 'ANN',
    # 'Ableton Live',
    'Activations': None,
    'Aeriolod': None,
    # 'AlexNet',
    'Analytics  Competition': None,
    # 'Animating Landscape',
    # 'Anomaly Detection',
    'ArtBreeder': 'Artbreeder',
    # 'Artbreeder',
    # 'AttnGAN',
    # 'AutoML',
    # 'B3D',
    # 'BASNet',
    # 'Background Removal',
    # 'Bayesian',
    # 'BigGAN',
    # 'BodyPix',
    # 'Boltzmann Machine',
    # 'CAD',
    # 'CMA-ES',
    # 'CMT',
    # 'CNN',
    # 'CPPN',
    # 'CV',
    'Camera': None,
    # 'Chatbot',
    # 'Classification',
    'Classifier': 'Classification',
    # 'Clustering',
    # 'Colorization',
    'Contentless': None,
    # 'Corpus-based synthesis',
    # 'CycleGAN',
    # 'DCGAN',
    # 'DDSP',
    # 'DL',
    # 'DLIB',
    # 'DeOldify',
    # 'Decision Tree',
    'Deep Fakes': 'DeepFake',
    # 'Deep Painterly Harmonization',
    # 'DeepDream',
    # 'DeepFake',
    # 'DeepFlow',
    # 'DenseCap',
    'Depth Map': None,
    # 'Detectron',
    'Detectron2': 'Detectron',
    'Device': None,
    'Discriminator': None,
    'Document Summarization': 'Summarization',
    # 'ESR-GAN',
    # 'Edge Detection',
    # 'Expression Detection',
    'Face Alignment': 'Face Detection',
    # 'Face Detection',
    # 'Face Recognition',
    'Face Tracking': 'Face Detection',
    'Face markers': 'Face Detection',
    'Face recognition': 'Face Recognition',
    'Facial Detection': 'Face Detection',
    'Facial Recognition': 'Face Recognition',
    # 'FastPhotoStyle',
    # 'Feature Mixing',
    'Feature vectors': None,
    'Featured Code Competition': None,
    'Featured Simulation Competition': None,
    'Featured prediction Competition': None,
    # 'Federated Learning',
    # 'Few Shot Animation',
    'First Order Motion': None,
    # 'GAN',
    # 'GBM',
    # 'GPT',
    'GPT-2': 'GPT',
    # 'GRU',
    'Game of Life': None,
    # 'GauGAN',
    # 'Gaussian Mixture Model',
    # 'Genetic Algorithm',
    'Getting Started prediction Competition': None,
    # 'Gradient Ascent',
    # 'Gradient Boosting',
    # 'Gradient Smoothing',
    # 'Grannma Magnet',
    'Hardware': None,
    'Height map': None,
    'Heightfield': None,
    'Houdini': None,
    # 'Image Captioning',
    # 'Image Segmentation',
    # 'ImageJ',
    # 'ImageNet',
    # 'Inception',
    # 'Inpainting',
    'Interpolation': None,
    # 'IoT',
    'K-Means': 'K-means',
    # 'K-means',
    # 'KNN',
    # 'Ken Burns 3D',
    # 'Kolmogorov complexity',
    # 'LSTM',
    # 'Laplacian Pyramid',
    'Lenticular': 'Lenticular Printing',
    'Lenticular Print': 'Lenticular Printing',
    # 'Linear Regression',
    # 'Logistic Regression',
    'ML': None,
    # 'Machine Translation',
    'Machine translation': 'Machine Translation',
    'Magenta': None,
    # 'Markov Chain',
    'Markov Chains': 'Markov Chain',
    'Memory Mosaic': None,
    'Microphone': None,
    'Mixture Density Networks': 'MDN',
    'Multi-Domain Multi-Modality I2I translation': 'Image2Image',
    'Multi-Style Transfer': 'Style Transfer',
    # 'Music Transformer',
    # 'N-gram',
    # 'NER',
    # 'NLG',
    # 'NLP',
    # 'NLU',
    'NN': None,
    # 'NNS',
    # 'NSynth',
    'NSynth Super': 'NSynth',
    # 'Naive Bayes',
    'Nerual CA': 'Neural Cellular Automata',
    # 'Neural Cellular Automata',
    # 'Object Detection',
    # 'Occams razor',
    'Open Pose': 'OpenPose',
    # 'OpenCV',
    # 'OpenPose',
    # 'Optical Flow',
    'Optical flow': 'Optical Flow',
    'Perlin Noise': None,
    # 'Photogrammetry',
    'Photoshop': None,
    'Pix2Pix': 'Image2Image',
    'Pix2pix': 'Image2Image',
    'Pixel2style2pixel': 'Image2Image',
    'Playground Code Competition': None,
    'Playground prediction Competition': None,
    # 'Point Cloud',
    # 'PoseNet',
    # 'ProGAN',
    'Progressively Grown GAN': 'ProGAN',
    # 'Projective Non-negative Matrix Factorization',
    # 'Quantum Computer',
    # 'QuickDraw',
    # 'RL',
    # 'RNN',
    # 'Random Forest',
    # 'Raymarching',
    # 'ReLu',
    # 'Recommender',
    'Recruitment prediction Competition': None,
    'Rectifier': 'ReLu',
    # 'Regression',
    'Reinforcement Learning': 'RL',
    # 'ResNet',
    'Research Code Competition': None,
    'Research prediction Competition': None,
    'Resnet': 'ResNet',
    # 'SIFT',
    # 'SNGAN',
    # 'SOM',
    # 'Self-attention',
    'Semantic search': 'Semantic Search',
    # 'Sentiment Analysis',
    # 'Simplex Volume Maximization',
    # 'SinGAN',
    # 'SketchRNN',
    # 'Sparse Transformer',
    # 'Speech Recognition',
    # 'Speech to text',
    # 'Style Transfer',
    # 'StyleGAN',
    'StyleGAN2': 'StyleGAN',
    'StyleTransfer': 'Style Transfer',
    # 'Super Slo Mo',
    # 'Super-Resolution',
    'Super-resolution': 'Super-Resolution',
    'Superresolution': 'Super-Resolution',
    # 'Supervised Learning',
    'Support Vector Machines': 'SVM',
    'TensorFlow.js': None,
    'Tensorflow.js': None,
    'Text Classification': 'Classification',
    'Text To Speech': 'Text To Speech',
    'Text classification': 'Classification',
    'Text to Animation of Virtual Characters': 'Text to Animation',
    # 'Texture synthesis',
    # 'Transformer',
    # 'Translation',
    # 'U-Net',
    'U-net': 'U-Net',
    # 'UMAP',
    'Unsupervised learning': 'Unsupervised Learning',
    # 'VAE',
    # 'VGG',
    'VQ-VAE': 'VAE',
    # 'VR',
    'Video StyleTransfer': 'Style Transfer',
    # 'Voice Detection',
    'Voice detection': 'Voice Detection',
    # 'Watson Beat',
    # 'Wav2Lip',
    # 'WaveGAN',
    # 'Wavenet',
    'Weights': None,
    # 'Word2Vec',
    'advanced': None,
    'animals': None,
    'arts and entertainment': 'Arts and Entertainment',
    'astronomy': 'Astronomy',
    'audio data': None,
    'automobiles and vehicles': 'Automotive',
    'banking': 'Banking',
    'basketball': 'Sports',
    'bayesian statistics': None,
    'beginner': None,
    'bigquery': 'BigQuery',
    'binary classification': 'Classification',
    'biology': 'Biology',
    # 'cDCGAN',
    'california': None,
    'categorical data': None,
    'china': None,
    'classification': 'Classification',
    'clustering': 'CLustering',
    'cnn': 'CNN',
    'computer science': None,
    'computer vision': 'CV',
    'covid19': None,
    'cuml-UMAP': 'UMAP',
    'dailychallenge': None,
    'data analytics': None,
    'data cleaning': None,
    'data visualization': None,
    'decision tree': 'Decision Tree',
    'deep learning': 'DL',
    'deepflow': 'DeepFlow',
    'dimensionality reduction': 'Dimensionality Reduction',
    'diseases': 'Diseases',
    'e-commerce services': 'E-Commerce',
    'earth and nature': 'Earth and Nature',
    'employment': 'Employment',
    'ensembling': 'Ensembling',
    'environment': 'Environment',
    'exploratory data analysis': 'Exploratory Data Analysis',
    'feature engineering': 'Feature Engineering',
    'finance': 'Finance',
    'forestry': 'Forestry',
    'games': 'Games',
    'gan': 'GAN',
    'genetics': 'Genetics',
    'geospatial analysis': 'Geospatial Analysis',
    'gpu': None,
    'gradient boosting': 'Gradient Boosting',
    'health': 'Healthcare',
    'healthcare': 'Healthcare',
    'image data': None,
    'india': None,
    'intermediate': None,
    'jobs and career': None,
    'k-means': 'K-means',
    'keras': 'Keras',
    'languages': None,
    'learn': None,
    'lightgbm': 'Gradient Boosting',
    'linear regression': 'Linear Regression',
    'linguistics': None,
    'logistic regression': 'Logistic Regression',
    'lstm': 'LSTM',
    'medicine': 'Healthcare',
    'microcontroller': None,
    'model comparison': 'Model Comparison',
    'model explainability': 'Model Explainability',
    'multiclass classification': 'Classification',
    'multilabel classification': 'Classification',
    'naive bayes': 'Naive Bayes',
    'neural networks': None,
    'nlp': 'NLP',
    'ofxSelfOrganizingMap': 'Self-organizing map',
    'openFrameworks': None,
    'optimization': None,
    'outlier analysis': 'Outlier Analysis',
    'pca': None,
    'physics': 'Physics',
    'pix2code': 'Pix2Code',
    'pix2pix': 'Image2Image',
    'plants': 'Plants',
    'pollution': 'Pollution',
    'puzzles': None,
    'python': None,
    'pytorch': None,
    # 'rGMIR',
    'random forest': 'Random Forest',
    'recommender systems': 'Recommender',
    'regression': 'Regression',
    'reinforcement learning': 'RL',
    'research': None,
    'rnn': 'RNN',
    'robotics': 'Robotics',
    'sampling': None,
    'signal processing': None,
    'simulations': None,
    'spaCy': 'NLP',
    'sports': 'Sports',
    'survey analysis': None,
    'svm': 'SVM',
    # 't-SNE',
    'tabular data': None,
    'tensorflow': None,
    'text data': None,
    'text mining': None,
    'time series analysis': 'Time Series Analysis',
    'tpu': None,
    'transfer learning': 'Transfer Learning',
    'utility script': None,
    'video games': 'Games',
    'xgboost': 'Xgboost',
}

def tag_equalizer(tags):
    tags = [tag_filter.get(x, x) for x in tags]
    tags = list(filter(None, tags))
    return tags

print(tag_equalizer(['tpu', 'rnn']))

['RNN']


In [27]:
# mapper to convert CSV to the mapping of Elasticsearch index
def mapper(row, style):
    '''
    mapper to adopt csv to db-schema

    "title"
    "summarization"
    "words"
    "sum_words"
    "link"
    "source"
    "category"
    "category_score"
    "subcategory"
    "subcategory_score"
    "tags"
    "kind"
    "ml_libs"
    "host"
    "license"
    "programming_language"
    "ml_score"
    "engagement_score"
    "date_project"
    "date_scraped"
    '''

    # kaggle competition mapping
    if style == 'kaggle_competition':
        return {
            'title': row['title'],
            'description': row['subtitle'] + row['description'],
            'link': row['link'],
            # 'category': '',
            # 'category_score': 0,
            # 'subcategory': '',
            # 'subcategory_score': 0,
            'tags': list(set(str_to_list(row['tags']) + str_to_list(row['type']))),
            'kind': ['Project', '(Competition)', '(Dataset)'],
            # 'ml_libs': str_to_list(row['ml_libs']),
            'host': 'www.kaggle.com',
            # 'license': row['license'],
            # 'programming_language': row['type'],
            # 'ml_score': 0,
            'engagement_score': row['teams_score'],
            'date_project': datetime.strptime(row['date_closed'], "%Y-%m-%d %H:%M:%S") if 'date_closed' in row else '',
            # 'date_scraped': datetime.strptime(row['scraped_at'], "%Y-%m-%d %H:%M:%S"),
            # 'ml_terms': row['ml_terms'],
            # 'score_raw': json.dumps({'views': row['views'], 'votes': row['votes'], 'score_private': row['score_private'], 'score_public': row['score_public']}),
        }
    
    # kaggle dataset mapping
    if style == 'kaggle_dataset':
        return {
            'title': row['title'],
            'description': row['subtitle'] + row['description'],
            'link': row['link'],
            # 'category': '',
            # 'category_score': 0,
            # 'subcategory': '',
            # 'subcategory_score': 0,
            'tags': list(set(str_to_list(row['tags']) + str_to_list(row['type']))),
            'kind': ['Project', '(Dataset)'],
            # 'ml_libs': str_to_list(row['ml_libs']),
            'host': 'www.kaggle.com',
            # 'license': row['license'],
            # 'programming_language': row['type'],
            # 'ml_score': 0,
            'engagement_score': row['teams_score'],
            'date_project': datetime.strptime(row['date_closed'], "%Y-%m-%d %H:%M:%S") if 'date_closed' in row else '',
            # 'date_scraped': datetime.strptime(row['scraped_at'], "%Y-%m-%d %H:%M:%S"),
            # 'ml_terms': row['ml_terms'],
            # 'score_raw': json.dumps({'views': row['views'], 'votes': row['votes'], 'score_private': row['score_private'], 'score_public': row['score_public']}),
        }
    
    # kaggle notebook mapping
    if style == 'kaggle_notebook':
        return {
            'title': row['title'],
            'description': row['description'],
            'link': row['link'],
            # 'category': '',
            # 'category_score': 0,
            # 'subcategory': '',
            # 'subcategory_score': 0,
            'tags': list(set(str_to_list(row['tags']) + str_to_list(row['tags']))),
            'kind': ['Project', '(Notebook)'],
            'ml_libs': str_to_list(row['ml_libs']),
            'host': 'www.kaggle.com',
            'license': row['license'],
            'programming_language': row['type'],
            'ml_score': row['ml_detected'],
            'engagement_score': row['score_views'] if 'score_views' in row else None,
            'date_project': datetime.strptime(row['date'], "%Y-%m-%d %H:%M:%S") if row['date'] != '' else None,
            'date_scraped': datetime.strptime(row['scraped_at'], "%Y-%m-%d %H:%M:%S") if row['scraped_at'] != '' else None,
            # 'ml_terms': row['ml_terms'],
            # 'score_raw': json.dumps({'views': row['views'], 'votes': row['votes'], 'score_private': row['score_private'], 'score_public': row['score_public']}),
        }

    # github mapping
    if style == 'github':
        title = row['name'] if row['name'] != '' else row['title']
        title = title.replace('-',' ').replace('_',' ').strip()
        cat_score = 1 if row['industry'] != '' else 0
        subcat_score = 1 if row['type'] != '' else 0
        #tags = row['ml_tags'] if len(row['ml_tags']) > 0 else ''
        return {
            'title': title,
            'description': row['description2'],
            'link': row['link'],
            'category': row['industry'],
            'category_score': cat_score,
            'subcategory': row['type'],
            'subcategory_score': subcat_score,
            'tags': str_to_list(row['ml_tags']),
            'kind': 'Project',
            'ml_libs': str_to_list(row['ml_libs']),
            'host': 'www.github.com',
            'license': row['license'],
            'programming_language': row['language_primary'],
            'ml_score': row['ml_detected'],
            'engagement_score': row['stars_score'],
            'date_project': datetime.strptime(row['pushed_at'], "%Y-%m-%d %H:%M:%S"),
            'date_scraped': datetime.strptime(row['scraped_at'], "%Y-%m-%d %H:%M:%S"),
            # 'ml_terms': row['keywords'],
            # 'score_raw': json.dumps({'stars': row['stars'], 'contributors': row['contributors']}),
        }

    # mlart mapping
    if style == 'mlart':
        title = row['Title'] if row['Title'] != '' else row['title']
        cat_score = 1 if row['Theme'] != '' else 0
        subcat_score = 1 if row['Medium'] != '' else 0
        return {
            'title': title,
            'description': row['subtitle'],
            'link': row['url'],
            'category': str_to_list(row['Theme']),
            'category_score': cat_score,
            'subcategory': str_to_list(row['Medium']),
            'subcategory_score': subcat_score,
            'tags': str_to_list(row['Technology']),
            'kind': 'Showcase',
            # 'ml_libs': [],
            'host': 'mlart.co',
            # 'license': '',
            # 'programming_language': '',
            # 'ml_score': row['ml_detected'],
            # 'engagement_score': 0,
            'date_project': datetime.strptime(row['Date'], "%Y-%m-%d"),
            'date_scraped': datetime.strptime(row['scraped_at'], "%Y-%m-%d %H:%M:%S"),
            # 'score_raw': json.dumps({'days_since_featured': row['Days Since Featured']}),
        }

    # thecleverprogrammer
    if style == 'tcp':
        return {
            'title': row['title'],
            'description': row['description'],
            'link': row['link'],
            # 'category': '',
            # 'category_score': 0,
            # 'subcategory': '',
            # 'subcategory_score': 0,
            'tags': str_to_list(row['ml_tags']),
            'kind': 'Project',
            'ml_libs': str_to_list(row['ml_libs']),
            'host': 'thecleverprogrammer.com',
            # 'license': '',
            'programming_language': 'Python',
            'ml_score': row['ml_score'],
            # 'engagement_score': 0,
            'date_project': datetime.strptime(row['date'], "%Y-%m-%d %H:%M:%S"),
            'date_scraped': datetime.strptime('2020-12-20', "%Y-%m-%d"),
            # 'score_raw': json.dumps({'days_since_featured': row['Days Since Featured']}),
        }

    return None

In [28]:
# test gpu usage
import torch
torch.cuda.is_available()

True

In [29]:
# summarization loop

In [43]:
# loop to transform data row-wise
def transform_loop(csv_in, csv_format, subfolder, quit=0, overwrite=False, inplace=True):
    
    with open(csv_in, encoding='utf-8') as csvfile:
        
        # let's store converted csv to temp-folder for analysis
        csv_out = '../data/database/csv/'
        json_out = '../data/database/json/'
        json_out_item = '../data/database/json/'+subfolder
        create_folder(json_out_item)
        df = pd.DataFrame()

        # readCSV = csv.reader(csvfile, delimiter=';')
        readCSV = csv.DictReader(csvfile, delimiter=';')
        # next(readCSV, None)  # skip the headers
        
        i = j = 0
        out = []
        
        for row in readCSV:
            # print(row)
            row = mapper(row, csv_format)
            
            # check if file already exists
            link = row['link']
            md5 = hashlib.md5(link.encode("utf-8")).hexdigest()
            
            json_fp = json_out_item + md5 + '.json'
            if not os.path.isfile(json_fp) or overwrite == True:
                if overwrite == True:
                    old = load_data(json_fp, fromJson=True)
                else:
                    old = None
                
                print(i, row['link'])
                item_start = time.time()

                # clean title & description
                row['title'] = clean_text(row['title'])
                text = row['description'] = clean_text(row['description'])
                words = row['words'] = word_count(text)
                sentences = row['sentences'] = sentence_count(text)

                # create summarization
                if words > 200 and sentences > 1:
                    print('summarize')
                    
                    # nltk
                    if old == None or not 'sum_nltk' in old:
                        start = time.time()
                        row['sum_nltk'] = nltk_count(text, word_count=200)
                        end = time.time()
                        dur = round(end-start,3)

                        row['sum_nltk_words'] = word_count(row['sum_nltk'])
                        row['sum_nltk_runtime'] = dur
                        print('done (nltk)', dur, 'sec')
                    
                    # t5
                    if old == None or not 'sum_t5' in old:
                        start = time.time()
                        row['sum_t5'] = t5(text)
                        end = time.time()
                        dur = round(end-start,3)

                        row['sum_t5_words'] = word_count(row['sum_t5'])
                        row['sum_t5_runtime'] = dur
                        print('done (t5)', dur, 'sec')
                
                # detect language
                s = row['description'] if 'description' in row and row['description'] != '' else row['title']
                lang = lingo(s, simple=False)
                row['language_code'] = lang['code']
                row['language'] = lang['language']
                row['language_score'] = lang['probability']

                # equalizer
                if 'programming_language' in row and row['programming_language'] == 'Python notebook':
                    row['programming_language'] = 'Jupyter Notebook'
                    
                if 'license' in row:
                    if row['license'] == 'Apache 2.0':
                        row['license'] = 'Apache-2.0'
                    if row['license'] == 'Learn more about GitHub Sponsors':
                        row['license'] = None
                    if row['license'] == 'Unlicense':
                        row['license'] = None
                        
                row['tags'] = tag_equalizer(row['tags'])
                

                # convert datetime to string
                if 'date_project' in row:
                    row['date_project'] = str(row['date_project'])
                if 'date_scraped' in row:
                    row['date_scraped'] = str(row['date_scraped'])
                    
                # runtime
                item_end = time.time()
                item_dur = round(item_end-item_start, 3)
                row['runtime'] = item_dur

                df = df.append(row, ignore_index=True)

                # json encode
                #out.append(row)
                
                if overwrite == True and inplace==True:
                    row = {**old, **row}
                    drop = ['score']
                    for key in drop:
                        if key in row:
                            row.pop(key)
                    # restore category, subcategory and runtime
                    if row['category'] == '' and 'category' in old:
                        row['category'] = old['category']
                        row['category_score'] = old['category_score']
                    if row['subcategory'] == '' and 'subcategory' in old:
                        row['subcategory'] = old['subcategory']
                        row['subcategory_score'] = old['subcategory_score']
                    if row['runtime'] == '' and 'runtime' in old:
                        row['runtime'] = old['runtime']
                            
                #print(row)
                #sys.exit()
                
                store_data(row, json_fp, toJson=True)
                print(json_fp)
                j += 1

            #print(i, row['link'])
            i += 1

            # keep count of # rows processed
            if i % 100 == 0:
                print(i)

            if quit != 0 and i >= quit:
                break

        # store parsed csv
        #fp = csv_in.split('/')[-1]
        #df.to_csv(csv_out + fp, sep=';', index=False)
        #path = json_out + fp
        #path = path.replace('.csv', '.json')
        #store_data(out, path, toJson=True)
        
        print('DONE parsed', i, 'items')

In [44]:
# run the loop

#transform = ['ka_c', 'ka_cn', 'ka_d', 'ka_dn', 'ma', 'gh', 'tcp', 'bc']
transform = ['ka_c', 'ka_cn', 'ma', 'gh', 'tcp', 'bc']
#transform = ['ma']

datasets = {
    # kaggle competitions
    'ka_c': {
        'csv_in': '../data/database/kaggle_competitions_correlated_01.csv',
        'csv_format': 'kaggle_competition',
    },
    # kaggle competitions notebooks
    'ka_cn': {
        'csv_in': '../data/database/kaggle_competitions_01_original.csv',
        'csv_format': 'kaggle_notebook',
    },
    # kaggle datasets
    'ka_d': {
        'csv_in': '../data/database/kaggle_datasets_correlated_01.csv',
        'csv_format': 'kaggle_dataset',
    },
    # kaggle datasets notebooks
    'ka_dn': {
        'csv_in': '../data/database/kaggle_datasets_01_original.csv',
        'csv_format': 'kaggle_notebook',
    },
    # mlart
    'ma': {
        'csv_in': '../data/database/mlart_01_original.csv',
        'csv_format':'mlart',
    },
    # github
    'gh': {
        'csv_in': '../data/database/db_04_analyzed_v02.csv',
        'csv_format': 'github',
    },
    # thecleverprogrammer
    'tcp': {
        'csv_in': '../data/database/thecleverprogrammer_01_original.csv',
        'csv_format': 'tcp',
    },
    # blobcity
    'bc': {
        'csv_in': '../data/database/blobcity_02_analyzed.csv',
        'csv_format': 'github',
    },
}

    
for key in transform:
    print(key)
    item = datasets[key]
    transform_loop(item['csv_in'], item['csv_format'], key+'/', overwrite=True)

ka_c
0 https://www.kaggle.com/c/20-newsgroups-ciphertext-challenge
summarize
../data/database/json/ka_c/07554a25ba5010fc437c588c02637782.json
1 https://www.kaggle.com/c/3d-object-detection-for-autonomous-vehicles
summarize
../data/database/json/ka_c/998dc92361462a193b1eee10472bc19e.json
2 https://www.kaggle.com/c/abstraction-and-reasoning-challenge
summarize
../data/database/json/ka_c/f1b8d78bbe5c3441abb14e95f265c987.json
3 https://www.kaggle.com/c/aerial-cactus-identification
summarize
../data/database/json/ka_c/10172da4e494109a2ed53419081a1096.json
4 https://www.kaggle.com/c/airbnb-recruiting-new-user-bookings
../data/database/json/ka_c/c4455229a9996e08978b53afd46e7fea.json
5 https://www.kaggle.com/c/airbus-ship-detection
summarize
../data/database/json/ka_c/a09ba09d429792c7deec1b662bb71827.json
6 https://www.kaggle.com/c/alaska2-image-steganalysis
summarize
../data/database/json/ka_c/647c8d2facd6bc052331da47849e6ed2.json
7 https://www.kaggle.com/c/allstate-claims-severity
../data/da

../data/database/json/ka_c/3d35b75c2ebd07971f12b4134167178d.json
66 https://www.kaggle.com/c/facebook-v-predicting-check-ins
summarize
../data/database/json/ka_c/39bcebdca682806d5d19fdbb82fa457d.json
67 https://www.kaggle.com/c/FacebookRecruiting
../data/database/json/ka_c/922cb868b7bce032f1b4bcb8b83cfb6c.json
68 https://www.kaggle.com/c/facial-keypoints-detection
../data/database/json/ka_c/3e8014290b0c66e49df19ae5006f90d2.json
69 https://www.kaggle.com/c/favorita-grocery-sales-forecasting
../data/database/json/ka_c/70326b2b01e875a3eedb646d694900f9.json
70 https://www.kaggle.com/c/flavours-of-physics-kernels-only
summarize
../data/database/json/ka_c/214cdb59fbf0df4fd87cdc36706a4fae.json
71 https://www.kaggle.com/c/flower-classification-with-tpus
summarize
../data/database/json/ka_c/7654365a4e65e4c949241d8d31e8c65f.json
72 https://www.kaggle.com/c/forest-cover-type-kernels-only
summarize
../data/database/json/ka_c/b1756523af13a69ea6cb499042391ee1.json
73 https://www.kaggle.com/c/forest-

../data/database/json/ka_c/f26016d0466158fed9010f0e16afddfb.json
131 https://www.kaggle.com/c/m5-forecasting-uncertainty
summarize
../data/database/json/ka_c/d1ad59124515d2f423209210f16c54af.json
132 https://www.kaggle.com/c/march-machine-learning-mania-2017
../data/database/json/ka_c/e103b32aaeea3888aa4a55459013d110.json
133 https://www.kaggle.com/c/melbourne-university-seizure-prediction
summarize
../data/database/json/ka_c/cfaa48b95bb56c33acb01eabe3639b58.json
134 https://www.kaggle.com/c/mens-machine-learning-competition-2018
summarize
../data/database/json/ka_c/9a6b17e044d8f03cc816cf365ee03cdf.json
135 https://www.kaggle.com/c/mens-machine-learning-competition-2019
summarize
../data/database/json/ka_c/00a474022f69b0bf64c8fa1136dc1ee0.json
136 https://www.kaggle.com/c/mercari-price-suggestion-challenge
summarize
../data/database/json/ka_c/d208af0c992558f906f911da1b2fd9e4.json
137 https://www.kaggle.com/c/mercedes-benz-greener-manufacturing
summarize
../data/database/json/ka_c/7d92d

../data/database/json/ka_c/3a08373a4445f764a226b5766bc8ac1f.json
196 https://www.kaggle.com/c/two-sigma-connect-rental-listing-inquiries
summarize
../data/database/json/ka_c/fd3376077e08958b41fa55c88056403d.json
197 https://www.kaggle.com/c/two-sigma-financial-modeling
summarize
../data/database/json/ka_c/b04f1133f07bff6fca4989a54be24a8c.json
198 https://www.kaggle.com/c/two-sigma-financial-news
summarize
../data/database/json/ka_c/44643fdaf17de0393848b49efb013145.json
199 https://www.kaggle.com/c/ultrasound-nerve-segmentation
../data/database/json/ka_c/b68bcd3ee8f86550db46cd818eda36c2.json
200
200 https://www.kaggle.com/c/understanding_cloud_organization
summarize
../data/database/json/ka_c/779fb2f20c9f3c122cecd42a5ec4ff52.json
201 https://www.kaggle.com/c/unimelb
summarize
../data/database/json/ka_c/5caf935c730bbb3d87a756098c1357e5.json
202 https://www.kaggle.com/c/vsb-power-line-fault-detection
../data/database/json/ka_c/f0f6a5def6dd3e2f9b15bd943d54ed29.json
203 https://www.kaggle.c

../data/database/json/ka_cn/d7639adda948c31705d90368d0b311b1.json
42 https://www.kaggle.com/abhinand05/in-depth-guide-to-convolutional-neural-networks
summarize
../data/database/json/ka_cn/e6f710de5baaebe3efad9bbe26fa51db.json
43 https://www.kaggle.com/aleksandradeis/arial-cactus-identification-with-pytorch-and-vgg16
../data/database/json/ka_cn/b81a1cd1c7860f8cd9d42bf141f23a53.json
44 https://www.kaggle.com/arjunrao2000/beginners-guide-efficientnet-with-keras
../data/database/json/ka_cn/57b445bf9895d6e4987ecb0bf786c008.json
45 https://www.kaggle.com/artgor/detecting-cactus-with-kekas
../data/database/json/ka_cn/9605df1578ddcdf94e5caff77831ad43.json
46 https://www.kaggle.com/ateplyuk/keras-starter-efficientnet
../data/database/json/ka_cn/a93e15858e7dc716c37eb84e6e7d633d.json
47 https://www.kaggle.com/ateplyuk/pytorch-efficientnet
../data/database/json/ka_cn/6167c484c9dc86fb267e15d984479dd9.json
48 https://www.kaggle.com/ateplyuk/starter-pytorch
../data/database/json/ka_cn/88b4af19ebc0af

...

298 https://github.com/LSSTC-DSFP/LSSTC-DSFP-Sessions
../data/database/json/bc/fe73085f0e581d921e2b3bcb8a193f54.json
299 https://github.com/lyhue1991/spark_tutorial
../data/database/json/bc/4412cc00e2b554fe804d64e425a50ca4.json
300
300 https://github.com/m2dsupsdlclass/lectures-labs
../data/database/json/bc/e7ca9db7ed7a36759b6047e4e1457c33.json
301 https://github.com/maartenbreddels/ipyvolume
../data/database/json/bc/4ae1638ee756ef2275079ff813152d2c.json
302 https://github.com/maartenbreddels/ipywebrtc
../data/database/json/bc/3a8bc385b9132f76f0aab02ff5d7206e.json
303 https://github.com/MaayanLab/clustergrammer-widget
../data/database/json/bc/fb8acf22f1cd7c051d94005dd8690fe7.json
304 https://github.com/MaayanLab/Zika-RNAseq-Pipeline
../data/database/json/bc/06ecd7b70dbfa9da4194c82be7d7666e.json
305 https://github.com/man-group/notebooker
../data/database/json/bc/7dbbc53cf13b70e9722a59d35ed6d1a8.json
306 https://github.com/mandli/numerical-methods-pdes
../data/database/json/bc/bb43ebe25

375 https://github.com/patrickvonplaten/notebooks
../data/database/json/bc/1c06291bd93e7e3a1eb27f510ed4dddf.json
376 https://github.com/pbugnion/gmaps
../data/database/json/bc/57cc30fecb91a120d6ee1f5970f90d92.json
377 https://github.com/PegasusWang/notebooks
../data/database/json/bc/1e1a978c17ff630b9b01eec4dd1b4e92.json
378 https://github.com/peterroelants/notebooks
../data/database/json/bc/f056283a4fc257e4e8bdf0aadaa06cb7.json
379 https://github.com/pgmpy/pgmpy_notebook
../data/database/json/bc/5a3138acb4c40efe1a714b08bfd80ef1.json
380 https://github.com/piegu/fastai-projects
../data/database/json/bc/ce86aa06bdc1a96127a973d25194e7ea.json
381 https://github.com/pierpaolo28/Data-Visualization
../data/database/json/bc/e729ad383dc332c9ec75c39d6b88becd.json
382 https://github.com/pierrelux/notebooks
../data/database/json/bc/2bd2b9246e8afc17df686280401bde4f.json
383 https://github.com/PipelineAI/katacoda-notebooks
../data/database/json/bc/497c08b6f0299b7573a0b91ff9e4051f.json
384 https://gi

../data/database/json/bc/7800e74d57c2cae54667d48a36c8081e.json
448 https://github.com/sanzgiri/deeplearning.ai
../data/database/json/bc/92ce589d0fb1f150bceeccacb56e4c7e.json
449 https://github.com/sat28/githubcommit
../data/database/json/bc/d5c9356efba552e4b4b600d6d0a4ecff.json
450 https://github.com/scipy/scipy-cookbook
../data/database/json/bc/7da9ee048dd63701b4fe23d276f9e86e.json
451 https://github.com/scoutbee/pytorch-nlp-notebooks
../data/database/json/bc/cacc7ab360f20627d3048c0919d88158.json
452 https://github.com/sean-parent/notebook
../data/database/json/bc/dc5b8d3d5341738d1835a11f697f85dd.json
453 https://github.com/seranus/faceswap-notebooks
../data/database/json/bc/f3485f6c0924c8d88a75b251b0d2e2da.json
454 https://github.com/sergiogama/notebook
../data/database/json/bc/cae0db40e47ad3123b410df6881288a2.json
455 https://github.com/sgugger/Deep-Learning
../data/database/json/bc/5249d5514c30f56e43b3c30fcb37f642.json
456 https://github.com/shakedzy/notebooks
../data/database/json

../data/database/json/bc/2be5402f61f59d52920d60124bc27e2b.json
520 https://github.com/watson-developer-cloud/assistant-improve-recommendations-notebook
../data/database/json/bc/47e4628d41a3f2f35e8d2519d23441f8.json
521 https://github.com/wenmin-wu/jupyter-tabnine
../data/database/json/bc/6b60cb2122cc05a3eeb0d19e7a20490a.json
522 https://github.com/wesm/pydata-book
../data/database/json/bc/bb488a09b19025fafa0ccbd4ecec28e5.json
523 https://github.com/wilfredinni/python-cheatsheet
../data/database/json/bc/563d91784cb62dcafc7c6cbbffacff63.json
524 https://github.com/willb/fraud-notebooks
../data/database/json/bc/9013a17f0df52aa58452a981891e540b.json
525 https://github.com/WittyOrator/JupyterNotebook
../data/database/json/bc/3b280b9286f653faa7fdd5abaaa328b6.json
526 https://github.com/wy-ei/notebook
../data/database/json/bc/1771c9f25304aad3ee86cb0ee15b204b.json
527 https://github.com/yandexdataschool/mlhep2019
../data/database/json/bc/01bfab1498f513ecc56415c3a296a182.json
528 https://github

LangDetectException: No features in text.

In [32]:
# zero shot categorization is computational intense
# so let's keep it out from the loop and process it seperatly

In [33]:
print(cat)
print(subcat)

['Accommodation & Food', 'Accounting', 'Agriculture', 'Banking & Insurance', 'Biotechnological & Life Sciences', 'Construction & Engineering', 'Economics', 'Education & Research', 'Emergency & Relief', 'Finance', 'Government and Public Works', 'Healthcare', 'Justice, Law and Regulations', 'Manufacturing', 'Media & Publishing', 'Miscellaneous', 'Physics', 'Real Estate, Rental & Leasing', 'Utilities', 'Wholesale & Retail']
['Failure', 'Food', 'Fraud', 'General', 'Genomics', 'Insurance and Risk', 'Judicial Applied', 'Life-sciences', 'Machine Learning', 'Maintenance', 'Management and Operations', 'Marketing', 'Material Science', 'Physical', 'Policy and Regulatory', 'Politics', 'Preventative and Reactive', 'Quality', 'Real Estate', 'Rental & Leasing', 'Restaurant', 'Retail', 'School', 'Sequencing', 'Social Policies', 'Student', 'Textual Analysis', 'Tools', 'Tourism', 'Trading & Investment', 'Transportation', 'Valuation', 'Water & Pollution', 'Wholesale']


In [46]:
# classification

folder = '../data/database/json/'
subfolder = os.listdir(folder)
#print(subfolder)

#transform = ['ka_c', 'ka_cn', 'ka_d', 'ka_dn', 'ma', 'gh', 'tcp', 'bc']
transform = ['ka_c', 'ka_cn', 'ma', 'gh', 'tcp', 'bc']
#transform = ['ma']

recreate_category = False
save = True
categorzie_t5 = False
categorize_nltk = True
categorize_fallback = True

quit = 0
i = j = 0
for item in subfolder:
    print('folder', item)
    fp = os.path.join(folder, item)
    if os.path.isdir(fp) and item in transform:
        print('###')
        print(item)
        files = os.listdir(fp)
        print('files in folder:', len(files))
        for file in files:
            row = load_data(os.path.join(folder, item, file), fromJson=True)
            #print(row)
            
            print('row:', i, 'item:', j, 'link:', row['link'], 'file:', file)
            
            # zero shot categorization
            if not 'category' in row or row.get('category') == '' or recreate_category == True:
                print('categorize')
                start = time.time()
                j += 1

                # create category and subcategory from t5
                if 'sum_t5' in row and row['sum_t5'] != '' and categorzie_t5 == True:
                    s = row['sum_t5']
                    res = categorize(s, cat)
                    #row['t5_category_raw'] = res
                    c = row['t5_category'] = res['category']
                    c_score = row['t5_category_score'] = res['score']
                    row['t5_category_runtime'] = res['runtime']
                    print('t5 category', res['runtime'], 'sec')

                    res = categorize(s, subcat)
                    #row['t5_subcategory_raw'] = res
                    sc = row['t5_subcategory'] = res['category']
                    sc_score = row['t5_subcategory_score'] = res['score']
                    row['t5_subcategory_runtime'] = res['runtime']
                    print('t5 subcategory', res['runtime'], 'sec')
                else:
                    print('t5 skipped')

                # create category and subcategory from nltk
                if 'sum_nltk' in row and row['sum_nltk'] != '' and categorize_nltk == True:
                    s = row['sum_nltk']
                    res = categorize(s, cat)
                    #print(res)
                    #row['nltk_category_raw'] = res
                    c = row['nltk_category'] = res['category']
                    c_score = row['nltk_category_score'] = res['score']
                    row['nltk_category_runtime'] = res['runtime']
                    print('nltk category', res['runtime'], 'sec')

                    res = categorize(s, subcat)
                    #print(res)
                    #row['nltk_subcategory_raw'] = res
                    sc = row['nltk_subcategory'] = res['category']
                    sc_score = row['nltk_subcategory_score'] = res['score']
                    row['nltk_subcategory_runtime'] = res['runtime']
                    print('nltk subcategory', res['runtime'], 'sec')
                else:
                    print('nltk skipped')

                # create category and subcategory from title or description if not already done
                if categorize_fallback == True and not 't5_category' in row and not 'nltk_category' in row:
                    if len(row['description']) > 0:
                        s = row['description']
                        res = categorize(s, cat)
                        #row['description_category_raw'] = res
                        c = row['description_category'] = res['category']
                        c_score = row['description_category_score'] = res['score']
                        row['description_category_runtime'] = res['runtime']
                        print('description category', res['runtime'], 'sec')

                        res = categorize(s, subcat)
                        #row['description_subcategory_raw'] = res
                        sc = row['description_subcategory'] = res['category']
                        sc_score = row['description_subcategory_score'] = res['score']
                        row['description_subcategory_runtime'] = res['runtime']
                        print('description subcategory', res['runtime'], 'sec')
                    else:
                        s = row['title']
                        if s != '':
                            res = categorize(s, cat)
                            #row['title_category_raw'] = res
                            c = row['title_category'] = res['category']
                            c_score = row['title_category_score'] = res['score']
                            row['title_category_runtime'] = res['runtime']
                            print('title category', res['runtime'], 'sec')

                            res = categorize(s, subcat)
                            #row['title_subcategory_raw'] = res
                            sc = row['title_subcategory'] = res['category']
                            sc_score = row['title_subcategory_score'] = res['score']
                            row['title_subcategory_runtime'] = res['runtime']
                            print('title subcategory', res['runtime'], 'sec')
                        else:
                            print('nothing found to categorize')
                            c = sc = ''
                            c_score = sc_score = 0
                            j -= 1

                row['category'] = c
                row['category_score'] = c_score
                row['subcategory'] = sc
                row['subcategory_score'] = sc_score

                end = time.time()
                dur = round(end-start, 3)
                row['runtime_cat'] = dur
                
                fp = os.path.join(folder, item, file)
                if save == True:
                    store_data(row, fp, toJson=True)
                else:
                    print('NOT SAVED')
                    print(row)
            
            i += 1
            
            if i%100 == 0:
                print(i)
            
            if quit!= 0 and i >= quit:
                break
    if quit!= 0 and i >= quit:
                break
            
print('DONE parsed', i, 'items')

folder bc
###
bc
files in folder: 541
row: 0 item: 0 link: https://github.com/HazyResearch/cs145-notebooks-2016 file: 003eb1b987f35ed6d88b1c7930e6057f.json
row: 1 item: 0 link: https://github.com/marsggbo/deeplearning.ai_JupyterNotebooks file: 004a0f102a03862a74da782e6a796f80.json
categorize
t5 skipped
nltk skipped
title category 6.505 sec
title subcategory 4.751 sec
row: 2 item: 1 link: https://github.com/davidbp/lxmls-notebooks file: 004a915abfded4001d49385a09f77f19.json
categorize
t5 skipped
nltk skipped
title category 2.722 sec
title subcategory 3.852 sec
row: 3 item: 2 link: https://github.com/jamesdbrock/learn-you-a-haskell-notebook file: 0065cbc76fd0832c7b3fc8ef5baed767.json
row: 4 item: 2 link: https://github.com/ageron/tf2_course file: 00708ba1c3a32ce45749ebcb633e8c5a.json
row: 5 item: 2 link: https://github.com/InsightDataLabs/ipython-notebooks file: 00b2db9f2524a95318f16eed6e605dd0.json
categorize
t5 skipped
nltk skipped
title category 2.846 sec
title subcategory 3.819 sec
r

title category 2.375 sec
title subcategory 3.534 sec
row: 74 item: 11 link: https://github.com/d2l-ai/d2l-book file: 22c958fbc28addccdf31a339fb688df7.json
row: 75 item: 11 link: https://github.com/ysyisyourbrother/SYSU_Notebook file: 239a0f9778f3b4c3d603a509fdd5312f.json
row: 76 item: 11 link: https://github.com/Alireza-Akhavan/rnn-notebooks file: 23d36f2684274e246b3377e85acbad53.json
row: 77 item: 11 link: https://github.com/rtidatascience/connected-nx-tutorial file: 2479d3699677cfbd582c51fc81d35bdc.json
row: 78 item: 11 link: https://github.com/thibo73800/tensorflow2.0-examples file: 253fed001f9b271e993a34a58f5c9abb.json
row: 79 item: 11 link: https://github.com/falloutdurham/beginners-pytorch-deep-learning file: 255767b67037b0153ba095c81a05a89c.json
row: 80 item: 11 link: https://github.com/dformoso/sklearn-classification file: 25fc184a3f7b7c0755cb9aa959326929.json
row: 81 item: 11 link: https://github.com/qutip/qutip-notebooks file: 278ec77fa645c2f1710cf32ff8af6a08.json
row: 82 ite

title category 4.537 sec
title subcategory 6.847 sec
row: 151 item: 15 link: https://github.com/OTRF/notebooks-forge file: 4dfea1bae4708d4317f7e570aa7322c0.json
row: 152 item: 15 link: https://github.com/uwdata/visualization-curriculum file: 4e4a4fa94d836e0555b197ae6a5743da.json
row: 153 item: 15 link: https://github.com/IBM/nodejs-in-notebooks file: 4e686c249c8efbd71dbfdb18386e4338.json
row: 154 item: 15 link: https://github.com/velocyto-team/velocyto-notebooks file: 502b31c0cd0f584eda981914ef8360c7.json
categorize
t5 skipped
nltk skipped
title category 4.447 sec
title subcategory 4.61 sec
row: 155 item: 16 link: https://github.com/kaleko/CourseraML file: 5067d72f4fca5b73c8c5a5c3f1edee8e.json
row: 156 item: 16 link: https://github.com/timsainb/python_spectrograms_and_inversion file: 506ef9f98987c54b0d315037a5a988ea.json
row: 157 item: 16 link: https://github.com/GeostatisticsLessons/GeostatisticsLessonsNotebooks file: 518989b27d43d3ae7b293493115c2982.json
row: 158 item: 16 link: https

title category 2.743 sec
title subcategory 3.831 sec
row: 216 item: 24 link: https://github.com/gaoxuesong/KerasNotebook file: 729d0290ce7b53c367b979bd5a8b76fe.json
row: 217 item: 24 link: https://github.com/azer/notebook file: 72a8cfb46d095370b5bd86fc3eb7408a.json
row: 218 item: 24 link: https://github.com/ledmaster/notebooks_tutoriais file: 733b401a95f63056d62b248b2a49fc8d.json
row: 219 item: 24 link: https://github.com/biocore/American-Gut file: 733d3661df3fbe2f3da6919dcb4c5981.json
row: 220 item: 24 link: https://github.com/planet-os/notebooks file: 733e84a3ffd9a565930cac5d5ce482e5.json
row: 221 item: 24 link: https://github.com/CODAIT/covid-notebooks file: 73cca01b71841d8b9dd3d502aed6688f.json
row: 222 item: 24 link: https://github.com/neuwangmeng/Keras_Jupyter_Notebooks file: 73f3edadceb2beb5c8b64324d23589ae.json
categorize
t5 skipped
nltk skipped
title category 3.425 sec
title subcategory 4.617 sec
row: 223 item: 25 link: https://github.com/d2l-ai/1day-notebooks file: 757014471f

title category 2.826 sec
title subcategory 4.357 sec
row: 290 item: 33 link: https://github.com/InsightSoftwareConsortium/SimpleITK-Notebooks file: 8e07512583080008bbfbbad7595123f5.json
row: 291 item: 33 link: https://github.com/codingforentrepreneurs/Notebooks file: 8ed65267940acc24b073cf0b9fee9af9.json
row: 292 item: 33 link: https://github.com/Yorko/python_intro file: 8ef7fc02e4c7f514177949953c0387a2.json
row: 293 item: 33 link: https://github.com/willb/fraud-notebooks file: 9013a17f0df52aa58452a981891e540b.json
categorize
t5 skipped
nltk skipped
title category 2.516 sec
title subcategory 3.535 sec
row: 294 item: 34 link: https://github.com/ageron/handson-ml file: 9094c555ec82851cbd778860a314c1fd.json
row: 295 item: 34 link: https://github.com/hardmaru/pytorch_notebooks file: 90e6d9f3028284ef22a32035b4df8fc9.json
row: 296 item: 34 link: https://github.com/aws/amazon-sagemaker-examples file: 90f8151eacad20743aecb34c18fc39f9.json
row: 297 item: 34 link: https://github.com/root-project

...

row: 3189 item: 57 link: https://www.kaggle.com/bibuml/beating-cvxtz-very-good-code-0-38-to-0-42 file: f4394152c1d80586412805fd262d9497.json
row: 3190 item: 57 link: https://www.kaggle.com/robertsturrock/prediction-using-fivethirtyeight-elo-ratings file: f4399cdfe4daabfd98ba2a3c5e971da4.json
row: 3191 item: 57 link: https://www.kaggle.com/davidmezzetti/trec-covid-submission file: f4429ee71e1c98f06bdf9e7a93bf8f0d.json
row: 3192 item: 57 link: https://www.kaggle.com/zinovadr/nfl-tableau-analysis file: f496472e172929d0036e0d6d0186e6fa.json
row: 3193 item: 57 link: https://www.kaggle.com/patilpramod2157/transfer-learning-using-inception-resnet-v2 file: f4c05d6b4fec14a0c6acc938b6d5fdf3.json
row: 3194 item: 57 link: https://www.kaggle.com/hiralmshah/robot-sensor-eda-fe-and-prediction-improvement file: f4c558fd0a91f70fc2d2b3d9027e13a3.json
row: 3195 item: 57 link: https://www.kaggle.com/dferhadi/covid-19-predictions-growth-factor-and-calculus file: f520673f28ed3ba218f106ca75bf65d1.json
row: 3

row: 3324 item: 57 link: https://mlart.co/item/compose-violin-music-with-prompts-trained-with-a-combination-of-a-cnn-and-rnn_-and-performed file: 1dc09112b4378a773aa3305cb90f6c70.json
row: 3325 item: 57 link: https://mlart.co/item/using-a-gan-to-generate-architecture-plans file: 1dd40c1440fd5a16962c9a9770b6acea.json
row: 3326 item: 57 link: https://mlart.co/item/generate-lyrics-with-an-vae-trained-with-seven-different-styles file: 1ee5ccf71bb506dfcb73f0483ed331cc.json
row: 3327 item: 57 link: https://mlart.co/item/classifies-drawings-and-matches-them-with-professional-graphics file: 1f16868747c36ca1e5172e6cd5c216f0.json
row: 3328 item: 57 link: https://mlart.co/item/finger-pressure-sensors_-a-breath-pressure-and-embouchure-sensor-on-the-mouthpiece-mapped-with-an-ann-to-generate-sound-via-a-microcontroller file: 21082913c712832481061a513ba6dd8b.json
row: 3329 item: 57 link: https://mlart.co/item/a-collaborative-art-tool-for-mixing-the-latent-space-of-different-gan-models-to-produce-new-

row: 3458 item: 57 link: https://mlart.co/item/train-a-gan-on-nike-shoes_-generate-an-image_-and-produce-a-shoe-collection-according-to-the-generated-image file: 7b6e139722b49806abaa0115a464a857.json
row: 3459 item: 57 link: https://mlart.co/item/tom-cruise-deep-fake file: 7ba50ca6e84aaaebdf6fbe44aab74fa5.json
row: 3460 item: 57 link: https://mlart.co/item/visualize-the-latent-representation-of-a-grandmother-in-a-cnn file: 7c26c833cf6c4423b654552c27067079.json
row: 3461 item: 57 link: https://mlart.co/item/visualize-the-entire-imagenet-dataset-in-vertical-rows file: 7c28c16474d28f172b40e5338dbe195a.json
row: 3462 item: 57 link: https://mlart.co/item/a-computer-looks-at-itself-with-a-camera-and-translates-image-features-into-text-with-an-rnn file: 7c7e9cbfe72151ba794c06cf30e3ed6d.json
row: 3463 item: 57 link: https://mlart.co/item/gan-generated-japanese-wood-prints_-and-then-print-them-using-the-same-technique file: 7c8c27789fb46affb51c418198667564.json
row: 3464 item: 57 link: https://

row: 3590 item: 57 link: https://mlart.co/item/translate-online-maps-to-19th-century-maps-in-real-time-with-cyclegan file: d4cb44570a85b80ec7b17dd71543f73f.json
row: 3591 item: 57 link: https://mlart.co/item/stylegan-interpolation_-trained-on-karl-blossfeldt_s-herbarium-_1928 file: d5e358e4f01cb820de380d3d73740c63.json
row: 3592 item: 57 link: https://mlart.co/item/rnn-generated-fake-articles-seeded-by-real-world-article-titles file: d6ac2fc6448decbd3ca6c8241998505c.json
row: 3593 item: 57 link: https://mlart.co/item/visualize-the-layers-of-a-cnn-with-your-camera file: d87d2b9196fc0fbe8ade924a065208cf.json
row: 3594 item: 57 link: https://mlart.co/item/apply-cyclegan-to-a-sketch-of-the-fonthill-castle file: d8816f879487a1e32bdea87dec5eb904.json
row: 3595 item: 57 link: https://mlart.co/item/generate-chairs-with-a-gan-and-then-produce-the-generated-chairs file: d8ad874798bbc895ef84fb5d04543aa6.json
row: 3596 item: 57 link: https://mlart.co/item/a-stylegan-trained-on-eboy_s-database-of-p

row: 3712 item: 57 link: https://thecleverprogrammer.com/2020/05/23/data-science-project-book-recommendation-system-with-machine-learning/ file: 57a4eca27828c3e02d5f3bd071d10105.json
row: 3713 item: 57 link: https://thecleverprogrammer.com/2020/11/12/earthquake-prediction-model-with-machine-learning/ file: 584b19de20fa8800ca1e2a06f3d07d1a.json
row: 3714 item: 57 link: https://thecleverprogrammer.com/2020/12/01/keyword-extraction-with-python/ file: 59846dca6a174590f8178aa552943f49.json
row: 3715 item: 57 link: https://thecleverprogrammer.com/2020/06/12/sms-spam-detection-with-machine-learning/ file: 59e85ba018cc2bc0e2453fa16d94c684.json
row: 3716 item: 57 link: https://thecleverprogrammer.com/2020/09/06/lung-segmentation-with-machine-learning/ file: 5a47f5158b0d0755932c0931ca8fd23c.json
row: 3717 item: 57 link: https://thecleverprogrammer.com/2020/08/30/image-classification-with-tensorflow-in-machine-learning/ file: 5a7eea5788be2ef6e7c4aa69e5cf5c5c.json
row: 3718 item: 57 link: https://